In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import os
import pickle

# Define the AudioCNN class with adjustments for input size
class AudioCNN(nn.Module):
    def __init__(self, num_classes, input_size):
        super(AudioCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * (input_size // 8), 128)  # Adjusted for input size
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add a channel dimension if it's not there
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten the features
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Function to load data and extract features using Wav2Vec2
def load_data(dataset_path, data_type, sampling_rate=16000):
    data = []
    labels = []
    processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
    model_wav2vec2 = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h")

    for file_number in range(1, 51):
        filename = f"data_{file_number}_{data_type}.pkl"
        file_path = os.path.join(dataset_path, filename)
        if os.path.exists(file_path):
            with open(file_path, 'rb') as file:
                file_data = pickle.load(file)
                for waveform, label in file_data:
                    input_values = processor(waveform.squeeze().numpy(), return_tensors="pt", sampling_rate=sampling_rate).input_values
                    with torch.no_grad():
                        features = model_wav2vec2(input_values).last_hidden_state.mean(dim=1).squeeze()
                    data.append(features)
                    labels.append(label)

    labels = [x[0] for x in labels]
    return torch.stack(data), torch.tensor(labels)

# Load and prepare data
dataset_path = 'emi_dataset/'
train_data, train_labels = load_data(dataset_path, "train")
train_loader = DataLoader(TensorDataset(train_data, train_labels), batch_size=32, shuffle=True)

validate_data, validate_labels = load_data(dataset_path, "valid")
validate_loader = DataLoader(TensorDataset(validate_data, validate_labels), batch_size=32)

test_data, test_labels = load_data(dataset_path, "test")
test_loader = DataLoader(TensorDataset(test_data, test_labels), batch_size=32)

# Initialize the model
num_classes = 10  # Update this based on your dataset
input_size = 1024  # Feature dimension of Wav2Vec2
audio_cnn = AudioCNN(num_classes=num_classes, input_size=input_size)

# Training loop
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(audio_cnn.parameters(), lr=0.001)

num_epochs = 30  # You can adjust the number of epochs
for epoch in range(num_epochs):
    audio_cnn.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = audio_cnn(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Step {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")

    # Validation loop
    audio_cnn.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    for features, labels in validate_loader:
        outputs = audio_cnn(features)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    val_loss /= len(validate_loader)
    print(f'Epoch {epoch+1}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {(100 * correct / total):.2f}%')

# Test loop
audio_cnn.eval()
test_loss = 0.0
correct = 0
total = 0
for features, labels in test_loader:
    outputs = audio_cnn(features)
    loss = criterion(outputs, labels)
    test_loss += loss.item()
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
test_loss /= len(test_loader)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {(100 * correct / total):.2f}%')


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.e

Epoch 1/30, Step 1/59, Loss: 2.2995
Epoch 1/30, Step 11/59, Loss: 2.2884
Epoch 1/30, Step 21/59, Loss: 2.2930
Epoch 1/30, Step 31/59, Loss: 2.2871
Epoch 1/30, Step 41/59, Loss: 2.3117
Epoch 1/30, Step 51/59, Loss: 2.2874
Epoch 1, Validation Loss: 2.2515, Validation Accuracy: 15.83%
Epoch 2/30, Step 1/59, Loss: 2.2108
Epoch 2/30, Step 11/59, Loss: 2.2359
Epoch 2/30, Step 21/59, Loss: 2.2879
Epoch 2/30, Step 31/59, Loss: 2.1529
Epoch 2/30, Step 41/59, Loss: 2.1713
Epoch 2/30, Step 51/59, Loss: 2.2046
Epoch 2, Validation Loss: 2.0726, Validation Accuracy: 27.67%
Epoch 3/30, Step 1/59, Loss: 1.9454
Epoch 3/30, Step 11/59, Loss: 1.9668
Epoch 3/30, Step 21/59, Loss: 1.9172
Epoch 3/30, Step 31/59, Loss: 2.1058
Epoch 3/30, Step 41/59, Loss: 1.8933
Epoch 3/30, Step 51/59, Loss: 2.0971
Epoch 3, Validation Loss: 1.8483, Validation Accuracy: 36.50%
Epoch 4/30, Step 1/59, Loss: 1.5241
Epoch 4/30, Step 11/59, Loss: 1.7030
Epoch 4/30, Step 21/59, Loss: 1.7838
Epoch 4/30, Step 31/59, Loss: 1.9866
Epoc